In [1]:
!pip install transformers
!pip install psycopg2-binary

In [2]:
import os

import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from tensorflow.keras.preprocessing.sequence import pad_sequences
import psycopg2

In [3]:
def get_df(db_url) -> pd.DataFrame:
    conn = psycopg2.connect(db_url)
    curs = conn.cursor()
    curs.execute("SELECT * FROM training;")
    cols = [k[0] for k in curs.description]
    rows = curs.fetchall()
    df = pd.DataFrame(rows, columns=cols)
    curs.close()
    conn.close()
    return df

In [4]:
df = get_df(db_url)
df

NameError: name 'db_url' is not defined

In [ ]:
df.to_csv('training.csv')

In [ ]:
def bert_trainer(path: str, output_dir: str, epochs: int):
    max_len = 280
    if torch.cuda.is_available():
        print("CUDA Active")
        device = torch.device("cuda")
    else:
        print("CPU Active")
        device = torch.device("cpu")
    df = pd.read_csv(path)
    sentences = df["tweets"].values
    labels = df["labels"].values
    tokenizer = BertTokenizer.from_pretrained(
        'bert-base-uncased',
        do_lower_case=True,
    )
    inputs = [
        tokenizer.encode(sent, add_special_tokens=True) for sent in sentences
    ]
    inputs_ids = pad_sequences(
        inputs,
        maxlen=max_len,
        dtype="long",
        value=0,
        truncating="post",
        padding="post",
    )
    attention_masks = [
        [int(token_id != 0) for token_id in sent] for sent in inputs_ids
    ]
    train_inputs = torch.tensor(inputs_ids)
    train_labels = torch.tensor(labels)
    train_masks = torch.tensor(attention_masks)
    batch_size = 32
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(
        train_data,
        sampler=train_sampler,
        batch_size=batch_size,
    )
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=12,
        output_attentions=False,
        output_hidden_states=False,
    )
    if torch.cuda.is_available():
        model.cuda()
    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps,
    )
    loss_values = []
    print('\nTraining...')
    for epoch_i in range(1, epochs + 1):
        print(f"\nEpoch: {epoch_i}")
        total_loss = 0
        model.train()
        for step, batch in enumerate(train_dataloader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
            loss = outputs[0]
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_loss / len(train_dataloader)
        loss_values.append(avg_train_loss)
        print(f"Average Loss: {avg_train_loss}")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    print(f"\nSaving model to {output_dir}")
    model_to_save = model.module if hasattr(model, 'module') else model
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print("Finished!")

In [ ]:
bert_trainer("training.csv", 'saved_model', epochs=50)

CUDA Active


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


Training...

Epoch: 1
Average Loss: 1.7270801311884172

Epoch: 2
Average Loss: 1.0277408682383025

Epoch: 3
Average Loss: 0.8157347119771517

Epoch: 4
Average Loss: 0.6354136497546465

Epoch: 5
Average Loss: 0.4619215524349457

Epoch: 6
Average Loss: 0.3125769988848613

Epoch: 7
Average Loss: 0.22088987705035087

Epoch: 8
Average Loss: 0.1508576955932837

Epoch: 9
Average Loss: 0.0907790563427485

Epoch: 10
Average Loss: 0.06024135739948505

Epoch: 11
Average Loss: 0.03337203326802223

Epoch: 12
Average Loss: 0.020315636510554798

Epoch: 13
Average Loss: 0.013276399017717594

Epoch: 14
Average Loss: 0.010350581407785798

Epoch: 15
Average Loss: 0.006738486138578408

Epoch: 16
Average Loss: 0.005453948689720187

Epoch: 17
Average Loss: 0.004561087167344223

Epoch: 18
Average Loss: 0.004553920261036509

Epoch: 19
Average Loss: 0.003694614914890665

Epoch: 20
Average Loss: 0.003202843157431254

Epoch: 21
Average Loss: 0.002895827381274639

Epoch: 22
Average Loss: 0.002691035702203711

Ep

In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification


class FrankenBert:
    """
    Implements BertForSequenceClassification and BertTokenizer
    for binary classification from a saved model
    """

    def __init__(self, path: str):
        """
        If there's a GPU available, tell PyTorch to use the GPU.
        Loads model and tokenizer from saved model directory (path)
        """
        if torch.cuda.is_available():
            self.device = torch.device('cuda')
        else:
            self.device = torch.device('cpu')
        self.model = BertForSequenceClassification.from_pretrained(path)
        self.tokenizer = BertTokenizer.from_pretrained(path)
        self.model.to(self.device)

    def predict(self, text: str):
        """
        Makes a binary classification prediction based on saved model
        """
        inputs = self.tokenizer(
            text,
            padding=True,
            truncation=True,
            max_length=280,
            return_tensors='pt',
        ).to(self.device)
        output = self.model(**inputs)
        prediction = output[0].softmax(1)
        tensors = prediction.detach().cpu().numpy()
        result = np.argmax(tensors)
        confidence = tensors[0][result]
        return f"Rank: {result}, {100 * confidence:.2f}%"

In [ ]:
model = FrankenBert('saved_model')

In [ ]:
model.predict("Mickey Mouse is in the house")

'Rank: 0, 98.96%'

In [ ]:
model.predict("Cops gave me a speeding ticket for walking too fast")

'Rank: 0, 99.71%'

In [ ]:
model.predict("The cops showed up but didn't do anything")

'Rank: 1, 98.62%'

In [ ]:
model.predict("Cops held that guy with a neck hold")

'Rank: 2, 59.83%'

In [ ]:
model.predict("Cops punched me and pushed me to the ground")

'Rank: 2, 98.90%'

In [ ]:
model.predict("Cops hit her with a baton")

'Rank: 3, 92.43%'

In [ ]:
model.predict("Cops sprayed my mom with pepper spray")

'Rank: 4, 99.70%'

In [ ]:
model.predict("Cops shot rubber bullets at the crowd")

'Rank: 4, 94.11%'

In [ ]:
model.predict("Police used tear gas on a pedestrian for no reason")

'Rank: 4, 99.66%'

In [ ]:
model.predict("Cops killed that women")

'Rank: 5, 99.48%'

In [ ]:
model.predict("Yesterday I saw a policeman hit a poor person behind my house. I wonder whats going on")

'Rank: 1, 99.83%'

In [ ]:
model.predict("Man ran up to me and pepper sprayed me. I've called the cops, but they have not gotten themselves involved yet.")

'Rank: 4, 72.70%'